In [7]:
#%cd ~ && rm -rf NAS_LLM
!git clone -b features https://github.com/WpythonW/NAS_LLM.git
!uv pip install --system "numpy<2" optuna sympy
%cd NAS_LLM/Informer2020

Cloning into 'NAS_LLM'...
remote: Enumerating objects: 280, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 280 (delta 38), reused 70 (delta 18), pack-reused 175 (from 1)
Receiving objects: 100% (280/280), 3.31 MiB | 8.44 MiB/s, done.
Resolving deltas: 100% (122/122), done.
Using Python 3.12.12 environment at: /usr
Audited 3 packages in 91ms
/content/NAS_LLM/Informer2020/NAS_LLM/Informer2020


In [1]:
%cd NAS_LLM/Informer2020
from utils.tools import dotdict
from exp.exp_informer import Exp_Informer
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.dates import DateFormatter
from matplotlib.ticker import MaxNLocator
import os, optuna
import torch
import numpy as np
from optuna.storages import JournalStorage
from optuna.storages.journal import JournalFileBackend
import sys
import json
from pathlib import Path
from datetime import datetime

/content/NAS_LLM/Informer2020


In [3]:
!rm "/content/NAS_LLM/Informer2020/ETTh1;pred_len=48;OPTUNA.json"
!rm "/content/NAS_LLM/Informer2020/optunaETTh2.log"

In [4]:
class TrialLogger:
    def __init__(self, filepath='trials.json'):
        self.filepath = Path(filepath)
        self.trials = self._load()

    def _load(self):
        return json.loads(self.filepath.read_text()) if self.filepath.exists() else []

    def _save(self):
        self.filepath.write_text(json.dumps(self.trials, indent=2))

    def add(self, trial_number, params, measures):
        self.trials.append({
            'trial': trial_number,
            'timestamp': datetime.now().isoformat(),
            **{k: float(v) for k, v in measures.items()},
            'seq_len': params['seq_len'],
            'label_len': params['label_len'],
            'e_layers': params['e_layers'],
            'n_heads': params['n_heads'],
            'factor': params['factor']
        })
        self._save()

def objective(trial):
    args = dotdict()

    # Фиксированные параметры
    args.model = 'informer'
    args.data = 'custom'
    args.root_path = './'
    args.data_path = f'../data/{dataset_name}_prepared.csv'
    args.features = 'MS'
    args.target = 'OT'
    args.freq = 'h'
    args.checkpoints = './informer_checkpoints'
    args.d_model = 512
    args.d_layers = 2
    args.d_ff = 2048
    args.dropout = 0.05
    args.attn = 'prob'
    args.embed = 'timeF'
    args.activation = 'gelu'
    args.distil = True
    args.output_attention = False
    args.mix = True
    args.padding = 0
    args.batch_size = 32
    args.learning_rate = 0.0001
    args.loss = 'mse'
    args.lradj = 'type1'
    args.train_epochs = 6
    args.patience = 3
    args.num_workers = 0
    args.itr = 1
    args.des = 'optuna'
    args.use_amp = False

    # Используем обе GPU
    args.use_gpu = True
    args.use_multi_gpu = True
    args.devices = '0'
    args.device_ids = [0]
    args.gpu = 0

    # Перебираемые параметры с constraint
    args.seq_len = trial.suggest_categorical('seq_len', [96, 168, 336, 480, 720])
    args.label_len = trial.suggest_categorical('label_len', [24, 48, 96])

    # label_len должен быть <= seq_len
    if args.label_len > args.seq_len:
        raise optuna.TrialPruned()

    args.pred_len = pred_len ##########################CHANGE
    args.e_layers = trial.suggest_categorical('e_layers', [2, 3, 4, 6])
    args.n_heads = trial.suggest_categorical('n_heads', [8, 16])
    args.factor = trial.suggest_categorical('factor', [3, 5, 8, 10])

    args.enc_in = args.dec_in = 7
    args.c_out = 1
    args.detail_freq = args.freq
    args.freq = args.freq[-1:]

    exp = Exp_Informer(args)
    setting = f'informer_trial{trial.number}'

    exp.train(setting)
    mae_val, mse_val = exp.test(setting, flag='val')
    mae_test, mse_test = exp.test(setting, flag='test')

    measures = {'mae_val': mae_val, 'mse_val': mse_val, 'mae_test': mae_test, 'mse_test': mse_test}

    logger.add(trial.number, trial.params, measures)

    return mse_val

pred_len=48
dataset_name = 'ETTh1'
logger = TrialLogger(f'{dataset_name};pred_len={pred_len};OPTUNA.json')
storage = JournalStorage(JournalFileBackend('./optunaETTh2.log'))

study = optuna.create_study(direction='minimize',
                            storage=storage,
                            study_name=f'opt,pred_len={pred_len}',
                            load_if_exists=True)
study.optimize(objective, n_trials=35)

print(f'Best MSE: {study.best_value:.4f}')
print(f'Best params: {study.best_params}')

[I 2025-11-17 16:54:36,627] A new study created in Journal with name: opt,pred_len=48


Use GPU: cuda:0
train 11427
val 1695
test 3437
	iters: 100, epoch: 1 | loss: 0.1597323
	speed: 0.1745s/iter; left time: 356.5039s
	iters: 200, epoch: 1 | loss: 0.1545358
	speed: 0.1792s/iter; left time: 348.1585s
	iters: 300, epoch: 1 | loss: 0.1123137
	speed: 0.1786s/iter; left time: 329.0936s
Epoch: 1 cost time: 63.39132118225098
Epoch: 1, Steps: 357 | Train Loss: 0.1841034 Vali Loss: 0.0883855 Test Loss: 0.1254905
Validation loss decreased (inf --> 0.088385).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.1033071
	speed: 0.3884s/iter; left time: 654.7899s
	iters: 200, epoch: 2 | loss: 0.0988234
	speed: 0.1752s/iter; left time: 277.8923s
	iters: 300, epoch: 2 | loss: 0.1002247
	speed: 0.1764s/iter; left time: 262.0617s
Epoch: 2 cost time: 62.751425981521606
Epoch: 2, Steps: 357 | Train Loss: 0.1060498 Vali Loss: 0.2213896 Test Loss: 0.3487290
EarlyStopping counter: 1 out of 3
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.0677572
	

[I 2025-11-17 16:59:44,921] Trial 0 finished with value: 0.08848558366298676 and parameters: {'seq_len': 720, 'label_len': 48, 'e_layers': 2, 'n_heads': 8, 'factor': 3}. Best is trial 0 with value: 0.08848558366298676.


test shape: (107, 32, 48, 1) (107, 32, 48, 1)
test shape: (3424, 48, 1) (3424, 48, 1)
mse:0.12548169493675232, mae:0.28450819849967957
Use GPU: cuda:0
train 12051
val 1695
test 3437
	iters: 100, epoch: 1 | loss: 0.2099429
	speed: 0.0639s/iter; left time: 137.9258s
	iters: 200, epoch: 1 | loss: 0.2427274
	speed: 0.0642s/iter; left time: 131.9792s
	iters: 300, epoch: 1 | loss: 0.1543861
	speed: 0.0642s/iter; left time: 125.6603s
Epoch: 1 cost time: 24.137601375579834
Epoch: 1, Steps: 376 | Train Loss: 0.2132820 Vali Loss: 0.8472304 Test Loss: 0.3260701
Validation loss decreased (inf --> 0.847230).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.1188331
	speed: 0.1512s/iter; left time: 269.2018s
	iters: 200, epoch: 2 | loss: 0.0890351
	speed: 0.0641s/iter; left time: 107.7310s
	iters: 300, epoch: 2 | loss: 0.0899080
	speed: 0.0639s/iter; left time: 100.9775s
Epoch: 2 cost time: 24.047553062438965
Epoch: 2, Steps: 376 | Train Loss: 0.1027761 Vali Loss: 1.

[I 2025-11-17 17:01:40,515] Trial 1 finished with value: 0.8489022850990295 and parameters: {'seq_len': 96, 'label_len': 96, 'e_layers': 3, 'n_heads': 8, 'factor': 3}. Best is trial 0 with value: 0.08848558366298676.


...

test shape: (107, 32, 48, 1) (107, 32, 48, 1)
test shape: (3424, 48, 1) (3424, 48, 1)
mse:0.13441945612430573, mae:0.3002411127090454
Use GPU: cuda:0
train 11427
val 1695
test 3437
	iters: 100, epoch: 1 | loss: 0.1619980
	speed: 0.2530s/iter; left time: 516.8268s
	iters: 200, epoch: 1 | loss: 0.1281961
	speed: 0.2548s/iter; left time: 495.0350s
	iters: 300, epoch: 1 | loss: 0.1102947
	speed: 0.2544s/iter; left time: 468.9123s
Epoch: 1 cost time: 90.80943298339844
Epoch: 1, Steps: 357 | Train Loss: 0.1930113 Vali Loss: 0.3464646 Test Loss: 0.2016813
Validation loss decreased (inf --> 0.346465).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.1089338
	speed: 0.5898s/iter; left time: 994.4288s
	iters: 200, epoch: 2 | loss: 0.1167886
	speed: 0.2547s/iter; left time: 404.0162s
	iters: 300, epoch: 2 | loss: 0.1032326
	speed: 0.2543s/iter; left time: 377.9254s
Epoch: 2 cost time: 90.82446193695068
Epoch: 2, Steps: 357 | Train Loss: 0.1010544 Vali Loss: 0.286

[I 2025-11-17 20:37:06,575] Trial 33 finished with value: 0.15838752686977386 and parameters: {'seq_len': 720, 'label_len': 48, 'e_layers': 6, 'n_heads': 8, 'factor': 8}. Best is trial 22 with value: 0.07986032217741013.


test shape: (107, 32, 48, 1) (107, 32, 48, 1)
test shape: (3424, 48, 1) (3424, 48, 1)
mse:0.21468110382556915, mae:0.37702780961990356
Use GPU: cuda:0
train 11427
val 1695
test 3437
	iters: 100, epoch: 1 | loss: 0.1660734
	speed: 0.2526s/iter; left time: 515.9957s
	iters: 200, epoch: 1 | loss: 0.1550577
	speed: 0.2543s/iter; left time: 494.0930s
	iters: 300, epoch: 1 | loss: 0.1967475
	speed: 0.2541s/iter; left time: 468.2646s
Epoch: 1 cost time: 90.6634132862091
Epoch: 1, Steps: 357 | Train Loss: 0.1930062 Vali Loss: 0.2583492 Test Loss: 0.1808628
Validation loss decreased (inf --> 0.258349).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.1253295
	speed: 0.5892s/iter; left time: 993.3148s
	iters: 200, epoch: 2 | loss: 0.1084155
	speed: 0.2540s/iter; left time: 402.9046s
	iters: 300, epoch: 2 | loss: 0.0840336
	speed: 0.2543s/iter; left time: 377.8703s
Epoch: 2 cost time: 90.68884181976318
Epoch: 2, Steps: 357 | Train Loss: 0.1068986 Vali Loss: 0.192

[I 2025-11-17 20:48:24,508] Trial 34 finished with value: 0.11870692670345306 and parameters: {'seq_len': 720, 'label_len': 48, 'e_layers': 6, 'n_heads': 8, 'factor': 8}. Best is trial 22 with value: 0.07986032217741013.


test shape: (107, 32, 48, 1) (107, 32, 48, 1)
test shape: (3424, 48, 1) (3424, 48, 1)
mse:0.1877589374780655, mae:0.3369710445404053
Best MSE: 0.0799
Best params: {'seq_len': 720, 'label_len': 48, 'e_layers': 6, 'n_heads': 16, 'factor': 8}
